In [1]:
# First import useful libraries ( Ollama in Python)

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama


In [14]:
# define Model which we are using OLLAMA llama3.2

MODEL = "llama3.2"


In [15]:
# define a class Website, which send HTTP 'get' request to a specific URL 

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        # Using the BeautifulSoup library and requests for web scraping (retrieve HTML content)
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Retrieve the title of the webpage
        self.title = soup.title.string if soup.title else "No title found"

        # Remove irrelevant tags such as script, style, img, and input
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        # Extract the cleaned text from the body of the webpage
        self.text = soup.body.get_text(separator="\n", strip=True)


In [16]:
# define System and user prompts

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."



In [39]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide powerpoint slides describes of this website with bullet points. \
\n\n"
    user_prompt += website.text
    return user_prompt


In [40]:
def messages(website):  
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]



In [41]:
def summary_ppt(url):
    website = Website(url)

    # call ollama
    response = ollama.chat(model=MODEL, messages=messages(website))
    return response['message']['content']


In [44]:
# check by calling function

summary_ppt("https://bbc.com")

'This appears to be a news aggregator website, providing summaries of recent news articles from various sources around the world. Here is a sample summary article taken from the article that he has seen will become:\n\n"Ukraine crisis: Biden agrees with France that\nFrance and EU to the latest move, a deal on 24 hrs\nFrance and Ukraine\'s Macron to be blame for Europe \n\nUK \nFrench health minister says."'